In [57]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import folium
import pandas as pd

# Show the base graph network

In [60]:
root_dir = Path("").parent
file_path = Path("").parent / "base_graph_hazard_edges.gpkg"
path_csv = Path("").parent / "event_weights copy.csv"
gdf =  gpd.read_file(file_path, driver = "GPKG")


,u,v,key,ref,highway,oneway,reversed,length,rfid_c,rfid,...,bridge,lanes,name,maxspeed,junction,width,access,tunnel,osmid_original,geometry
0,301479950,7694372094,0,P444,tertiary,False,False,148.0,"[2816, 2818, 2857, 2859, 1964, 1965]",26,...,nan,nan,nan,nan,nan,nan,nan,nan,94930471,"LINESTRING (32.34856 -27.43878, 32.34832 -27.4..."
1,465172685,465172904,0,"['P522', 'P522-2']",secondary,False,"[False, True]",3016.0,"[3329, 3343, 19, 156, 161, 163, 166, 3495, 41,...",12,...,yes,nan,nan,nan,nan,nan,nan,nan,"[98700210, 1165570002, 98700198]","LINESTRING (32.27237 -27.04067, 32.27143 -27.0..."
2,465172685,1411601875,0,P522-2,secondary,False,False,1154.0,"[98, 1763, 134, 2822, 169, 106, 22, 2774, 2776...",13,...,nan,nan,nan,nan,nan,nan,nan,nan,98700198,"LINESTRING (32.23540 -27.04101, 32.23598 -27.0..."
3,465172685,1317736030,0,P435,tertiary,False,True,15188.0,"[6656, 6658, 3587, 6660, 6662, 3593, 6672, 667...",39,...,low_water_crossing,nan,nan,nan,nan,nan,nan,nan,"[195696985, 823343961, 544486156, 544486157]","LINESTRING (32.25148 -26.92411, 32.25048 -26.9..."
4,465172691,5267911961,0,P433,tertiary,False,True,5921.0,"[513, 515, 517, 519, 523, 525, 529, 531, 535, ...",25,...,nan,nan,nan,nan,nan,nan,nan,nan,545008607,"LINESTRING (32.11207 -27.11586, 32.11232 -27.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10910,1415130322,8187979822,0,nan,tertiary,False,False,2667.0,"[1260075, 1260077, 1260079, 1260081, 1260083, ...",16696,...,nan,nan,nan,nan,nan,nan,nan,nan,881002756,"LINESTRING (42.83291 0.36911, 42.83240 0.36899..."
10911,1415130588,4835116532,0,nan,tertiary,False,True,19979.0,"[1259521, 1259523, 1259525, 1259527, 1259529, ...",16694,...,yes,nan,nan,nan,nan,nan,nan,nan,"[1190264962, 1190264963, 885199854, 491384088,...","LINESTRING (42.77740 0.24994, 42.77735 0.24998..."
10912,1415132819,8185617347,0,nan,tertiary,False,True,9447.0,"[1259637, 1259647, 1258118, 1258120, 1258122, ...",16695,...,nan,nan,nan,nan,nan,nan,nan,nan,127971662,"LINESTRING (42.83932 0.43703, 42.83883 0.43769..."
10913,2005187111,8191343741,0,nan,primary,False,False,6589.0,"[1258380, 1259660, 1258382, 1258384, 1258386, ...",16697,...,nan,nan,Buulo Mareer - Jilib Highway,nan,nan,nan,nan,nan,647450158,"LINESTRING (42.84239 0.48576, 42.84112 0.48612..."


In [61]:
# import the scenario names and their respective weights:
scenarios_weights_df = pd.read_csv(path_csv)
number_of_scenarios = len(scenarios_weights_df)

columns_to_count = [name for name in scenarios_weights_df['name'].to_list() if name in gdf.columns]

weights_dict = {}
for _, row in scenarios_weights_df.iterrows():
    weights_dict[row['name']] = row['weight']





## Modify the dataframe

In [66]:
# Parametrization
threshold = 0.5  # would be nice to make this threshold into a slider.

color_on_probability = True

In [63]:


threshold = 0.5  # would be nice to make this threshold into a slider.
# Add a new column with the count of non-zero values across specified columns
gdf['count'] = gdf[columns_to_count].apply(lambda row: (row > threshold).sum(), axis=1)

for index, row in gdf.iterrows():
    total_weight = 0
    for col in columns_to_count:
        if row[col] > threshold:
            # Find the corresponding weight and add to total
            weight = weights_dict[col]
            total_weight += weight
    gdf.at[index, 'cum_pf'] = total_weight

temp = gdf[gdf['count'] > 0]




,u,v,key,ref,highway,oneway,reversed,length,rfid_c,rfid,...,name,maxspeed,junction,width,access,tunnel,osmid_original,geometry,count,cum_pf
212,4726239538,2145561630,0,N222,secondary,False,True,141252.0,"[98304, 98306, 98308, 98310, 98312, 98313, 983...",3370,...,nan,nan,nan,2,nan,nan,"[479541472, 665634467, 277215975, 479532844, 4...","LINESTRING (34.13248 -22.04119, 34.13081 -22.0...",1,0.002660
213,4726239538,4486637363,0,R441,tertiary,False,True,60012.0,"[22528, 22530, 22532, 22534, 22536, 22538, 225...",3007,...,nan,nan,nan,nan,nan,nan,"[650593120, 650593121, 865118090, 341846669]","LINESTRING (33.30126 -22.62187, 33.30118 -22.6...",1,0.002660
242,561058066,560487941,0,N221,secondary,False,False,4144.0,"[31104, 30080, 31106, 30082, 31108, 31493, 311...",403,...,208,nan,nan,nan,nan,nan,"[199231616, 199914163, 618555604, 618555603, 1...","LINESTRING (32.84510 -24.42512, 32.84424 -24.4...",1,0.002660
246,1895332932,2096868577,0,R412,secondary,False,False,68004.0,"[30720, 30722, 30724, 30726, 30728, 30732, 307...",434,...,405,nan,nan,nan,nan,nan,"[179185600, 179185605, 329437031, 243932168, 3...","LINESTRING (32.65031 -25.02710, 32.65063 -25.0...",2,0.005319
250,1810196335,4799853357,0,N1,trunk,False,True,25370.0,"[33792, 31745, 33794, 31747, 33796, 31749, 337...",448,...,nan,80,nan,nan,nan,nan,"[199700155, 199697540, 539652656, 539652657, 5...","LINESTRING (32.86452 -25.07204, 32.86242 -25.0...",1,0.002660


In [68]:
from branca.colormap import LinearColormap
 
# Create a Folium pop_diff_map centered around the mean of the geometry
center = [
    gdf['geometry'].centroid.y.mean(),
    gdf['geometry'].centroid.x.mean()
]
pop_diff_map = folium.Map(location=center, zoom_start=9, control_scale=True, tiles="cartodbpositron")
 
 
if color_on_probability:
    var = 'cum_pf'
else:
    var = "count"

vmin=(gdf[var]).min()
vmax=(gdf[var]).max()
# Classify values into ranges
ranges = [vmin, vmin+((vmax-vmin)/4), vmin+2*((vmax-vmin)/4), vmin+3*((vmax-vmin)/4) , vmax]

ranges=sorted(ranges)
# Create a colormap from green to green to red using the overall min and max values
colormap = LinearColormap(colors=['lightgreen', 'orange', 'darkred'],
                          vmin=0,
                          vmax=vmax
                          )
 
colormap.caption = 'Affected people-Optimal routes length difference (people x km)'
 

# Create a feature group for each quartile
for i, q in enumerate(list(ranges)):
    if i == 0:
        continue
    if i == len(ranges):
        break
    q_min = round(ranges[i-1], 1)
    q_max = round(ranges[i], 1)
 
    # Filter rows based on quartile
    filtered_rows = gdf[
        (gdf[var] >= q_min) &
        (gdf[var] <= q_max)
    ]
   

 
    # Add target_optimal_routes_with_hazard_gdf with created Colormap to feature_group
    for idx, row in filtered_rows.iterrows():
        value = row['count']
        cum_pf = row['cum_pf']
        # color = 'blue' if (row['count'] > 0)  else 'lightgrey' 

        if row[var] == 0:
            color = 'lightgrey'
        else:
            color = colormap(row[var])

 
        # Extracting coordinates from MultiLineString
        coordinates = []
        if row['geometry'].geom_type == 'MultiLineString':
            for line in row['geometry']:
                coords = [(coord[1], coord[0]) for coord in line.coords]
                coordinates.extend(coords)
        else:
            coordinates = [(coord[1], coord[0]) for coord in row['geometry'].coords]
        # Create a popup with data
        popup_content = f"Nb scenario disrupting {value}/{len(columns_to_count)}, cumulative pf: {cum_pf}"
        popup = folium.Popup(popup_content, max_width=300)
 
        folium.PolyLine(
            locations=coordinates,
            color=color,
            weight=3,
            opacity=1,
            popup=popup
        ).add_to(pop_diff_map)
 

#  from branca.colormap import LinearColormap


# colormap = branca.colormap.linear.YlOrRd_09.scale(0, 8500)
colormap = colormap.to_step(index=[1,2,3,4,5])
colormap.caption = 'Number of scenarios for which link is disrupted'
colormap.add_to(pop_diff_map)

#pop_diff_map.save(root_dir / "map_figure.html")
 
pop_diff_map